In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
import preprocess_utils.session2vec as sess2vec
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

Using TensorFlow backend.


#### sort order --> filter selection

price only --> Sort by Price

distance only --> Sort By Distance

rating only --> Sort By Rating

price and recommended --> Best Value

rating and recommended --> Focus on Rating

distance and recommended --> Focus on Distance

our recommendations --> Sort by Popularity

In [3]:
train_df = data.train_df('small', cluster='cluster_recurrent')

In [5]:
train_df[train_df.current_filters.notnull()]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
176,03P4VFKK12UO,325fafb5fa450,1541107500,54,clickout item,65685,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Resort|Hostal (ES)|3 Star,1306936|56482|2842358|6881276|65685|63259|65399|1320460|360566|376756|885381|60769|2141484|375876|829246|58909|360571|1298518|63542|71926|376076|57803|376181|1117192|359326,178|104|110|94|57|96|46|61|48|35|50|38|59|44|57|56|55|45|146|41|44|44|39|38|126,1.0
177,03P4VFKK12UO,325fafb5fa450,1541107538,55,clickout item,1320460,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Resort|Hostal (ES)|3 Star,1306936|56482|2842358|6881276|65685|63259|65399|1320460|360566|376756|885381|60769|2141484|375876|829246|58909|360571|1298518|63542|71926|376076|57803|376181|1117192|359326,178|104|110|94|57|96|46|61|48|35|50|38|59|44|57|56|55|45|146|41|44|44|39|38|126,1.0
445,0NA8E4AD2VY7,336d9a46b826f,1541078452,91,clickout item,1091180,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|1371070|1191128|320366|2589366|1373129|1091180|3404929|5813690|939106|940286|3183138|1370286|384511|5127144|2740100|939964|1327476|3961622|939114|1370246,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|116|29|79|38|42|28|47|28|50|67,1.0
446,0NA8E4AD2VY7,336d9a46b826f,1541078493,92,clickout item,1091180,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|1371070|1191128|320366|2589366|1373129|1091180|3404929|5813690|939106|940286|3183138|1370286|384511|5127144|2740100|939964|1327476|3961622|939114|1370246,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|116|29|79|38|42|28|47|28|50|67,1.0
466,0NA8E4AD2VY7,336d9a46b826f,1541079107,112,clickout item,3404929,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|1371070|1191128|320366|2589366|1373129|1091180|3404929|5813690|939106|940286|3183138|1370286|384511|5127144|2740100|939964|1327476|3961622|939114|1370246,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|116|29|79|38|42|28|47|28|50|67,1.0
487,0O1CTEP95YKQ,af793494b5f84,1541113474,19,clickout item,63051,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|8531|4169220|63051|397316|840726|2851635|1493341|8804492|62454|450176|102082|148954|8602|441851|63858|8516|8510248,59|66|78|86|87|90|90|92|93|98|99|100|106|108|111|111|112|120|122|123|125|136|138|140|142,1.0
488,0O1CTEP95YKQ,af793494b5f84,1541113509,20,clickout item,2851635,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|8531|4169220|63051|397316|840726|2851635|1493341|8804492|62454|450176|102082|148954|8602|441851|63858|8516|8510248,59|66|78|86|87|90|90|92|93|98|99|100|106|108|111|111|112|120|122|123|125|136|138|140|142,1.0
490,0O1CTEP95YKQ,af793494b5f84,1541113537,22,clickout item,8602,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|8531|4169220|63051|397316|840726|2851635|1493341|8804492|62454|450176|102082|148954|8602|441851|63858|8516|8510248,59|66|78|86|87|90|90|92|93|98|99|100|106|108|111|111|112|120|122|123|125|136|138|140|142,1.0
539,0Q3605EUBZG3,641a409c90fac,1541095202,44,clickout item,4041114,BR,"Rio de Janeiro, Brazil",mobile,Best Value,2303306|4041114|5830856|111534|109353|5723646|1208722|1255025|1670475|1055330|655691|2284048|104681|104809|130772|5822734|1668389|1771225|2342182|104800|1329092|9719914|2627243|3249257|109332,72|29|122|98|187|86|66|124|112|104|118|107|146|179|136|113|102|115|96|147|136|36|65|93|115,1.0
542,0Q3605EUBZG3,641a409c90fac,1541095238,47,clickout item,9112068,BR,"Rio de Janeiro, Brazil",mobile,Best Value,5016850|2068758|1771981|2303306|2733700|3189448|5849680|1179630|1319947|1770143|925941|926343|8428074|109331|829671|3906960|4094062|5

In [4]:
accom_df = data.accomodations_one_hot() #.fillna(0)

Loading accomodations one-hot...


In [15]:
train_df.shape

(221133, 13)

In [19]:
accom_df.head()

,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,Air Conditioning,Airport Hotel,Airport Shuttle,All Inclusive (Upon Inquiry),Balcony,Bathtub,Beach,Beach Bar,Beauty Salon,Bed & Breakfast,Bike Rental,Boat Rental,Body Treatments,Boutique Hotel,Bowling,Bungalows,Business Centre,Business Hotel,Cable TV,Camping Site,Car Park,Casa Rural (ES),Casino (Hotel),Central Heating,Childcare,Club Hotel,Computer with Internet,Concierge,Conference Rooms,Convenience Store,Convention Hotel,Cosmetic Mirror,Cot,Country Hotel,Deck Chairs,Design Hotel,Desk,Direct beach access,Diving,Doctor On-Site,Eco-Friendly hotel,Electric Kettle,Excellent Rating,Express Check-In / Check-Out,Family Friendly,Fan,Farmstay,Fitness,Flatscreen TV,Free WiFi (Combined),Free WiFi (Public Areas),Free WiFi (Rooms),Fridge,From 2 Stars,From 3 Stars,From 4 Stars,Gay-friendly,Golf Course,Good Rating,Guest House,Gym,Hairdresser,Hairdryer,Halal Food,Hammam,Health Retreat,Hiking Trail,Honeymoon,Horse Riding,Hostal (ES),Hostel,Hot Stone Massage,Hotel,Hotel Bar,House / Apartment,Hydrotherapy,Hypoallergenic Bedding,Hypoallergenic Rooms,Ironing Board,Jacuzzi (Hotel),Kids' Club,Kosher Food,Large Groups,Laundry Service,Lift,Luxury Hotel,Massage,Microwave,Minigolf,Motel,Nightclub,Non-Smoking Rooms,On-Site Boutique Shopping,Openable Windows,Organised Activities,Pet Friendly,Playground,Pool Table,Porter,Pousada (BR),Radio,Reception (24/7),Resort,Restaurant,Romantic,Room Service,Room Service (24/7),Safe (Hotel),Safe (Rooms),Sailing,Satellite TV,Satisfactory Rating,Sauna,Self Catering,Senior Travellers,Serviced Apartment,Shooting Sports,Shower,Singles,Sitting Area (Rooms),Ski Resort,Skiing,Solarium,Spa (Wellness Facility),Spa Hotel,Steam Room,Sun Umbrellas,Surfing,Swimming Pool (Bar),Swimming Pool (Combined Filter),Swimming Pool (Indoor),Swimming Pool (Outdoor),Szep Kartya,Table Tennis,Telephone,Teleprinter,Television,Tennis Court,Tennis Court (Indoor),Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms)
item_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5101,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,1,1,0,1,1,0,0,0,0,1,1,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,1,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,1,0,1,0,1,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1
5416,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,1
5834,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,1,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1
5910,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,0
6066,0,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,1,0,1,1,1,0,1,0,1,1,1,0,1,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,1,1,1,0,1,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,1,1,1,0,0,1,1,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,1,0,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1,0,1,0,0,1,1,1,1,0,1,0,1,1,1,0,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1


In [8]:
train_df[train_df.action_type == 'filter selection']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
125,03P4VFKK12UO,325fafb5fa450,1541107336,3,filter selection,Very Good Rating,US,"Bakersfield, USA",desktop,Very Good Rating,NaN,NaN,1.0
126,03P4VFKK12UO,325fafb5fa450,1541107366,4,filter selection,Hostal (ES),US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
127,03P4VFKK12UO,325fafb5fa450,1541107366,5,filter selection,Hotel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
128,03P4VFKK12UO,325fafb5fa450,1541107366,6,filter selection,Motel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
129,03P4VFKK12UO,325fafb5fa450,1541107366,7,filter selection,Resort,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
130,03P4VFKK12UO,325fafb5fa450,1541107366,8,filter selection,4 Star,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
131,03P4VFKK12UO,325fafb5fa450,1541107366,9,filter selection,5 Star,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
132,03P4VFKK12UO,325fafb5fa450,1541107368,10,filter selection,Hostal (ES),US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
133,03P4VFKK12UO,325fafb5fa450,1541107368,11,filter selection,Hotel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0
134,03P4VFKK12UO,325fafb5fa450,1541107368,12,filter selection,Motel,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,NaN,NaN,1.0


In [7]:
train_df[(train_df.user_id == '0O1CTEP95YKQ') & (train_df.session_id == 'af793494b5f84')]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
469,0O1CTEP95YKQ,af793494b5f84,1541113259,1,change of sort order,interaction sort button,CO,"Las Vegas, USA",mobile,NaN,NaN,NaN,1.0
470,0O1CTEP95YKQ,af793494b5f84,1541113276,2,filter selection,Hotel,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN,1.0
471,0O1CTEP95YKQ,af793494b5f84,1541113276,3,filter selection,Motel,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN,1.0
472,0O1CTEP95YKQ,af793494b5f84,1541113276,4,filter selection,Resort,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort,NaN,NaN,1.0
473,0O1CTEP95YKQ,af793494b5f84,1541113314,5,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Hotel|Motel|Resort|Breakfast Included,NaN,NaN,2.0
475,0O1CTEP95YKQ,af793494b5f84,1541113325,7,filter selection,Resort,CO,"Las Vegas, USA",mobile,Hotel|Motel,NaN,NaN,1.0
476,0O1CTEP95YKQ,af793494b5f84,1541113325,8,filter selection,Motel,CO,"Las Vegas, USA",mobile,Hotel,NaN,NaN,1.0
477,0O1CTEP95YKQ,af793494b5f84,1541113329,9,filter selection,Hotel,CO,"Las Vegas, USA",mobile,NaN,NaN,NaN,1.0
478,0O1CTEP95YKQ,af793494b5f84,1541113330,10,filter selection,Breakfast Included,CO,"Las Vegas, USA",mobile,Breakfast Included,NaN,NaN,1.0
479,0O1CTEP95YKQ,af793494b5f84,1541113350,11,filter selection,Free WiFi (Combined),CO,"Las Vegas, USA",mobile,Breakfast Included|Free WiFi (Combined),NaN,NaN,3.0


In [9]:
train_df[(train_df.action_type == 'interaction item image') & (train_df.current_filters.notnull())]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence


In [30]:
change_sort_filters = set(['sort by price', 'sort by distance', 'sort by rating', 'sort by popularity',
                    'focus on rating', 'focus on distance', 'best value'])

### Take the clickouts with some current_filters

In [8]:
clickouts = train_df[(train_df.action_type == 'clickout item')] # & train_df.current_filters.notnull()]
clickouts = clickouts[['user_id','session_id','current_filters','impressions']]
clickouts['filters_list'] = clickouts['current_filters'].str.lower().str.split('|')
clickouts['impress_list'] = clickouts['impressions'].str.split('|')
clickouts = clickouts.drop(['impressions','current_filters'], axis=1)
clickouts['impress_list'] = clickouts['impress_list'].apply(lambda x: list(map(int, x)))
clickouts['filters_list'] = clickouts['filters_list'].fillna('')

In [9]:
clickouts.head()

,user_id,session_id,filters_list,impress_list
13,00RL8Z82B2Z1,aff3928535f48,,"[3400638, 1253714, 3367857, 5100540, 1088584, 666916, 54833, 2922310, 9711560, 109038, 666856, 10077318, 1431482, 129343, 6339822, 6806806, 1041528, 109013, 3909420, 55088, 3095758, 109018, 54885, 1257342, 2595006]"
15,00RL8Z82B2Z1,aff3928535f48,,"[55109, 129343, 54824, 2297972, 109014, 1257342, 1031578, 109018, 1332971, 666916, 54833, 54885, 2237222, 10077318, 1166793, 9132132, 1474297, 3909420, 6622154, 55091, 8444418, 54804, 2050977, 666936, 55088]"
115,02SRUT1NQYH1,3599a6f709eab,,"[2795374, 5582964, 1088390, 2781070, 1258068, 1271962, 3184892, 148884, 3528776, 107183, 5156744, 107048, 1907333, 3370484, 6003326, 8436316, 9025316, 125181, 3861490, 131257, 4415954, 107162, 3143352, 6652864, 8118684]"
121,03K8AXBL4BX2,ec139e10b9238,,"[12693, 46363, 81657, 18448, 47687, 152913, 18417, 927627, 4147572, 819616, 18380, 47333, 18425, 1255377, 4043482, 1032816, 18388, 1216900, 106471, 106567, 12667, 84082, 1152188, 98737, 47410]"
122,03K8AXBL4BX2,ec139e10b9238,,"[12693, 46363, 81657, 18448, 47687, 152913, 18417, 927627, 4147572, 819616, 18380, 47333, 18425, 1255377, 4043482, 1032816, 18388, 1216900, 106471, 106567, 12667, 84082, 1152188, 98737, 47410]"


### Count the number of tags that the impressions have wrt to the filters

In [10]:
mlb = MultiLabelBinarizer(classes=accom_df.columns.str.lower())

In [17]:
# iterate over the clickouts and one-hot
print('Total interactions:', clickouts.shape[0])
result = np.zeros((clickouts.shape[0],25))
k = 0
for idx in tqdm(clickouts.index):
    filters = clickouts.at[idx, 'filters_list']
    # do not consider change-of-sort filters
    filters = set(filters).difference(change_sort_filters)
    # fix some wrong filters names
    if 'gay friendly' in filters:
        filters.remove('gay friendly')
        filters.add('gay-friendly')
    if 'internet (rooms)' in filters:
        filters.remove('internet (rooms)')
        filters.add('free wifi (rooms)')
    filters = list(filters)
    filters_len = len(filters)
    if filters_len > 0:
        # one-hot the filters
        filters_one_hot = mlb.fit_transform([filters])[0]
        # take the one-hot of the impressions tags
        impressions = clickouts.at[idx, 'impress_list']
        impressions_features_one_hot = accom_df.loc[impressions].values

        satisfaction_percentage = np.sum(filters_one_hot & impressions_features_one_hot, axis=1) / filters_len
        result[k, 0:len(impressions)] = satisfaction_percentage
    else:
        # there are only change-of-sort filters
        result[k, 0:len(impressions)] = 1
    k += 1

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['breakfast included'] will be ignored
  .format(sorted(unknown, key=str)))
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['kitchen'] will be ignored
  .format(sorted(unknown, key=str)))
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['hotel chain'] will be ignored
  .format(sorted(unknown, key=str)))
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['cheap'] will be ignored
  .format(sorted(unknown, key=str)))
/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['deals + beach (mx)'] will be ignored
  .format(sorted(u

In [ ]:
# add the 25 new columns
for i in range(25):
    clickouts['satisf_perc_{}'.format(i)] = result[:,i]

In [13]:
clickouts

,user_id,session_id,filters_list,impress_list,satisf_perc_0,satisf_perc_1,satisf_perc_2,satisf_perc_3,satisf_perc_4,satisf_perc_5,satisf_perc_6,satisf_perc_7,satisf_perc_8,satisf_perc_9,satisf_perc_10,satisf_perc_11,satisf_perc_12,satisf_perc_13,satisf_perc_14,satisf_perc_15,satisf_perc_16,satisf_perc_17,satisf_perc_18,satisf_perc_19,satisf_perc_20,satisf_perc_21,satisf_perc_22,satisf_perc_23,satisf_perc_24
176,03P4VFKK12UO,325fafb5fa450,"[very good rating, 5 star, 4 star, hotel, motel, resort, hostal (es), 3 star]","[1306936, 56482, 2842358, 6881276, 65685, 63259, 65399, 1320460, 360566, 376756, 885381, 60769, 2141484, 375876, 829246, 58909, 360571, 1298518, 63542, 71926, 376076, 57803, 376181, 1117192, 359326]",0.375000,0.125000,0.000000,0.125000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000
177,03P4VFKK12UO,325fafb5fa450,"[very good rating, 5 star, 4 star, hotel, motel, resort, hostal (es), 3 star]","[1306936, 56482, 2842358, 6881276, 65685, 63259, 65399, 1320460, 360566, 376756, 885381, 60769, 2141484, 375876, 829246, 58909, 360571, 1298518, 63542, 71926, 376076, 57803, 376181, 1117192, 359326]",0.375000,0.125000,0.000000,0.125000,0.125000,0.000000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000
445,0NA8E4AD2VY7,336d9a46b826f,"[free wifi (combined), breakfast included]","[1045290, 3135752, 1372961, 1369024, 3384308, 1371070, 1191128, 320366, 2589366, 1373129, 1091180, 3404929, 5813690, 939106, 940286, 3183138, 1370286, 384511, 5127144, 2740100, 939964, 1327476, 3961622, 939114, 1370246]",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
446,0NA8E4AD2VY7,336d9a46b826f,"[free wifi (combined), breakfast included]","[1045290, 3135752, 1372961, 1369024, 3384308, 1371070, 1191128, 320366, 2589366, 1373129, 1091180, 3404929, 5813690, 939106, 940286, 3183138, 1370286, 384511, 5127144, 2740100, 939964, 1327476, 3961622, 939114, 1370246]",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
466,0NA8E4AD2VY7,336d9a46b826f,"[free wifi (combined), breakfast included]","[1045290, 3135752, 1372961, 1369024, 3384308, 1371070, 1191128, 320366, 2589366, 1373129, 1091180, 3404929, 5813690, 939106, 940286, 3183138, 1370286, 384511, 5127144, 2740100, 939964, 1327476, 3961622, 939114, 1370246]",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
487,0O1CTEP95YKQ,af793494b5f84,"[breakfast included, sort by price]","[69086, 8533, 925367, 58949, 61049, 8581, 8517, 77520, 8531, 4169220, 63051, 397316, 840726, 2851635, 1493341, 8804492, 62454, 450176, 102082, 148954, 8602, 441851, 63858, 8516, 8510248]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
488,0O1CTEP95YKQ,af793494b5f84,"[breakfast included, sort by price]","[69086, 8533, 925367, 58949, 61049, 8581, 8517, 77520, 8531, 4169220, 63051, 397316, 840726, 2851635, 1493341, 8804492, 62454, 450176, 102082, 148954, 8602, 441851, 63858, 8516, 8510248]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [5]:
from extract_features.rnn.clickout_filters_satisfaction import ClickoutFiltersSatisfaction
feature = ClickoutFiltersSatisfaction()

In [6]:
aa = feature.join_to(train_df)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


clickout_filters_satisfaction feature read


In [7]:
aa.loc[[605]]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,satisf_perc_0,satisf_perc_1,satisf_perc_2,satisf_perc_3,satisf_perc_4,satisf_perc_5,satisf_perc_6,satisf_perc_7,satisf_perc_8,satisf_perc_9,satisf_perc_10,satisf_perc_11,satisf_perc_12,satisf_perc_13,satisf_perc_14,satisf_perc_15,satisf_perc_16,satisf_perc_17,satisf_perc_18,satisf_perc_19,satisf_perc_20,satisf_perc_21,satisf_perc_22,satisf_perc_23,satisf_perc_24
605,0T819ODEIRA4,7643893620070,1541084725,6,clickout item,5170834,CH,"Mühldorf am Inn, Germany",desktop,Free WiFi (Combined)|Restaurant,137842|162011|52319|5170834|9332768|6398694|1830581|157418|5608302|1073008|8156060|162013|4127242|2049255|1240136|8295690,59|95|102|109|60|61|119|65|85|85|80|65|54|46|99|72,1.0,1.0,0.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.0,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
aa[(aa.user_id == '0T819ODEIRA4') & (aa.session_id == '7643893620070')]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,satisf_perc_0,satisf_perc_1,satisf_perc_2,satisf_perc_3,satisf_perc_4,satisf_perc_5,satisf_perc_6,satisf_perc_7,satisf_perc_8,satisf_perc_9,satisf_perc_10,satisf_perc_11,satisf_perc_12,satisf_perc_13,satisf_perc_14,satisf_perc_15,satisf_perc_16,satisf_perc_17,satisf_perc_18,satisf_perc_19,satisf_perc_20,satisf_perc_21,satisf_perc_22,satisf_perc_23,satisf_perc_24
600,0T819ODEIRA4,7643893620070,1541084324,1,search for destination,"Mühldorf am Inn, Germany",CH,"Mühldorf am Inn, Germany",desktop,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
604,0T819ODEIRA4,7643893620070,1541084370,5,clickout item,5170834,CH,"Mühldorf am Inn, Germany",desktop,Free WiFi (Combined)|Restaurant,137842|162011|52319|5170834|9332768|6398694|1830581|157418|5608302|1073008|8156060|162013|4127242|2049255|1240136|8295690,59|95|102|109|60|61|119|65|85|85|80|65|54|46|99|72,1.0,1.0,0.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.0,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
605,0T819ODEIRA4,7643893620070,1541084725,6,clickout item,5170834,CH,"Mühldorf am Inn, Germany",desktop,Free WiFi (Combined)|Restaurant,137842|162011|52319|5170834|9332768|6398694|1830581|157418|5608302|1073008|8156060|162013|4127242|2049255|1240136|8295690,59|95|102|109|60|61|119|65|85|85|80|65|54|46|99|72,1.0,1.0,0.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.0,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,0T819ODEIRA4,7643893620070,1541085062,7,clickout item,5170834,CH,"Mühldorf am Inn, Germany",desktop,NaN,162011|52319|5170834|9332768|6398694|1830581|5608302|157418|1073008|8156060|162013|4127242|2049255|1240136|137842|8295690,109|110|109|90|94|149|85|108|115|91|90|79|75|167|105|87,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
impr = aa.loc[605].impressions.split('|')
impr = list(map(int,impr))
accom_df.loc[impr][['Restaurant', 'Free WiFi (Combined)']]

,Restaurant,Free WiFi (Combined)
item_id,,
137842,1,1
162011,0,0
52319,1,0
5170834,0,0
9332768,0,0
6398694,0,1
1830581,1,0
157418,1,0
5608302,0,1
